In [259]:
from datetime import datetime
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import udf, col, asc, desc, countDistinct
from pyspark.sql.functions import date_format, row_number, monotonically_increasing_id 
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType, TimestampType


In [11]:
import zipfile as zf

log_files = zf.ZipFile('data/log-data.zip', 'r')
log_files.extractall('data/log_data')
log_files.close()

song_files = zf.ZipFile('data/song-data.zip', 'r')
song_files.extractall('data')
song_files.close()

In [3]:
spark = SparkSession \
    .builder \
    .appName('DataLake') \
    .getOrCreate()

In [43]:
logDf = spark.read.json('data/log_data/*')
songDf = spark.read.json('data/song_data/*/*/*/*')

# logDf = spark \
#     .read \
#     .format('json') \
#     .option('inferSchema', 'true') \
#     .load('data/log_data/*')

In [64]:
songDf.show(1)
songDf.count()
songDf.printSchema()

+------------------+---------------+---------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+---------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARDR4AC1187FB371A1|           null|               |            null|Montserrat Caball...|511.16363|        1|SOBAYLL12A8C138AF9|Sono andati? Fing...|   0|
+------------------+---------------+---------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
only showing top 1 row

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- art

In [303]:
songs_schema = StructType([
    StructField('song_id', StringType(), nullable=False),
    StructField('title', StringType(), nullable=False),
    StructField('artist_id', StringType(), nullable=True),
    StructField('year', LongType(), nullable=True),
    StructField('duration', DoubleType(), nullable=True)
])

In [309]:
songs_rdd = songDf \
    .filter(col('song_id').isNotNull()) \
    .filter(col('title').isNotNull()) \
    .select('song_id', 'title', 'artist_id', 'year', 'duration') \
    .dropDuplicates(['song_id']) \
    .rdd

songs = spark.createDataFrame(songs_rdd, songs_schema)

songs.show(2)

71

In [291]:
songs \
    .write \
    .partitionBy('year', 'artist_id') \
    .mode('overwrite') \
    .parquet('tables/songs/songs.parquet')

In [101]:
songs_read = spark.read.parquet('tables/songs/songs.parquet')
songs_read.createOrReplaceTempView('songs')
spark.sql('SELECT * FROM songs WHERE title LIKE "Law%"').show()
spark.sql('SELECT COUNT(*) FROM songs').show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOOLYAZ12A6701F4A6|Laws Patrolling (...|AREBBGV1187FB523D2|   0|173.66159|
+------------------+--------------------+------------------+----+---------+

+--------+
|count(1)|
+--------+
|      71|
+--------+



In [103]:
artists_schema = StructType([
    StructField('artist_id', StringType(), nullable=False),
    StructField('name', StringType(), nullable=False),
    StructField('location', StringType(), nullable=True),
    StructField('latitude', DoubleType(), nullable=True),
    StructField('longitude', DoubleType(), nullable=True)
])

In [104]:
artists_rdd = songDf \
    .filter(col('artist_id').isNotNull()) \
    .filter(col('artist_name').isNotNull()) \
    .select('artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude') \
    .rdd

artists = spark.createDataFrame(artists_rdd, artists_schema)

artists.show(2)

+------------------+--------------------+-----------+--------+---------+
|         artist_id|                name|   location|latitude|longitude|
+------------------+--------------------+-----------+--------+---------+
|ARDR4AC1187FB371A1|Montserrat Caball...|           |    null|     null|
|AREBBGV1187FB523D2|Mike Jones (Featu...|Houston, TX|    null|     null|
+------------------+--------------------+-----------+--------+---------+
only showing top 2 rows



In [105]:
artists.write.parquet('tables/artists/artists.parquet')

In [107]:
logDf.show(1)
logDf.count()
logDf.printSchema()

+--------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+-------------+------+-------------+--------------------+------+
|  artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|         song|status|           ts|           userAgent|userId|
+--------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+-------------+------+-------------+--------------------+------+
|Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
+--------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+-------------+------+-------------+----

In [133]:
logDf.count()
logDf.groupBy('userId').count().show()

+------+-----+
|userId|count|
+------+-----+
|    51|   17|
|     7|    8|
|    15|  495|
|    54|   29|
|   101|   73|
|    11|    4|
|    29|  381|
|    69|   41|
|    42|  149|
|    73|  305|
|    87|    2|
|    64|    2|
|     3|    3|
|    30|  193|
|    34|   14|
|    59|    5|
|     8|   37|
|    28|    8|
|    22|    2|
|    85|  204|
+------+-----+
only showing top 20 rows



In [313]:
d = [
    {'name': 'Alice', 'age': 1},
    {'name': 'Alice', 'age': 2},
    {'name': 'Bob', 'age': 5}
]

df = spark.createDataFrame(d)
df.show()

w = Window.partitionBy('name').orderBy(col('age').desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)
df.withColumn('rn', row_number().over(w)).show()

df.dropDuplicates(['name']).show()


+---+-----+
|age| name|
+---+-----+
|  1|Alice|
|  2|Alice|
|  5|  Bob|
+---+-----+

+---+-----+---+
|age| name| rn|
+---+-----+---+
|  5|  Bob|  1|
|  2|Alice|  1|
|  1|Alice|  2|
+---+-----+---+

+---+-----+
|age| name|
+---+-----+
|  5|  Bob|
|  1|Alice|
+---+-----+



In [106]:
users_schema = StructType([
    StructField('user_id', LongType(), nullable=False),
    StructField('first_name', StringType(), nullable=True),
    StructField('last_name', StringType(), nullable=True),
    StructField('gender', StringType(), nullable=True),
    StructField('level', StringType(), nullable=True)
])

In [314]:
users_window = Window \
    .partitionBy('userId') \
    .orderBy(col('ts').desc()) \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

users_rdd = logDf \
    .filter(col('page') == 'NextSong') \
    .filter(col('userId').isNotNull()) \
    .dropDuplicates(['userId']) \
    .withColumn('num', row_number().over(users_window)) \
    .withColumn('user_id', col('userId').cast(LongType())) \
    .filter(col('num') == 1) \
    .select('user_id', 'firstName', 'lastName', 'gender', 'level') \
    .rdd
    
users = spark.createDataFrame(users_rdd, users_schema)

users.show(2)

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     51|      Maia|    Burke|     F| free|
|      7|    Adelyn|   Jordan|     F| free|
+-------+----------+---------+------+-----+
only showing top 2 rows



In [151]:
users.write.parquet('tables/users/users.parquet')

In [298]:
time_schema = StructType([
    StructField('start_time', TimestampType(), nullable=False),
    StructField('hour', IntegerType(), nullable=False),
    StructField('day', IntegerType(), nullable=False),
    StructField('week', IntegerType(), nullable=False),
    StructField('month', IntegerType(), nullable=False),
    StructField('year', IntegerType(), nullable=False),
    StructField('weekday', IntegerType(), nullable=False)
])

In [251]:
logDf.select('ts').show(10)

+-------------+
|           ts|
+-------------+
|1542241826796|
|1542242481796|
|1542242741796|
|1542247071796|
|1542252577796|
|1542253449796|
|1542253460796|
|1542260074796|
|1542260277796|
|1542260935796|
+-------------+
only showing top 10 rows



In [315]:
time_rdd = logDf \
    .select('ts') \
    .withColumn('timestamp', (col('ts') / 1000).cast(TimestampType())) \
    .dropDuplicates(['timestamp']) \
    .select(
        col('timestamp').alias('start_time'),
        hour('timestamp').alias('hour'),
        dayofmonth('timestamp').alias('day'),
        weekofyear('timestamp').alias('week'),
        month('timestamp').alias('month'),
        year('timestamp').alias('year'),
        date_format(col('timestamp'), 'F').cast(IntegerType()).alias('weekday')
    ) \
    .rdd

time = spark.createDataFrame(time_rdd, time_schema)

time.show(2)

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 16:12:...|  16| 15|  46|   11|2018|      3|
|2018-11-21 06:18:...|   6| 21|  47|   11|2018|      3|
+--------------------+----+---+----+-----+----+-------+
only showing top 2 rows



In [288]:
time \
    .write \
    .partitionBy('year', 'month') \
    .mode('overwrite') \
    .parquet('tables/time/time.parquet')

In [292]:
songplays_schema = StructType([
    StructField('songplay_id', LongType(), nullable=False),
    StructField('start_time', TimestampType(), nullable=False),
    StructField('user_id', LongType(), nullable=False),
    StructField('level', StringType(), nullable=True),
    StructField('song_id', StringType(), nullable=False),
    StructField('artist_id', StringType(), nullable=False),
    StructField('session_id', LongType(), nullable=True),
    StructField('location', StringType(), nullable=True),
    StructField('user_agent', StringType(), nullable=True),
    StructField('year', IntegerType(), nullable=False),
    StructField('month', IntegerType(), nullable=False)
])

In [295]:
clean_logDf = logDf \
    .filter(col('page') == 'NextSong')

clean_songDf = songDf \
    .filter(col('song_id').isNotNull()) \
    .filter(col('artist_id').isNotNull())

songplays_rdd = clean_songDf \
    .join(clean_logDf,
        (clean_songDf.title == clean_logDf.song)
            & (clean_songDf.artist_name == clean_logDf.artist)
            & (clean_songDf.duration == clean_logDf.length)
        , 'inner') \
    .withColumn('id', monotonically_increasing_id() + 1) \
    .withColumn('start_time', (col('ts') / 1000).cast(TimestampType())) \
    .withColumn('user_id', col('userId').cast(LongType())) \
    .withColumn('year', year('start_time')) \
    .withColumn('month', month('start_time')) \
    .select(
        'id'
        , 'start_time'
        , 'user_id'
        , 'level'
        , 'song_id'
        , 'artist_id'
        , 'sessionId'
        , 'location'
        , 'userAgent'
        , 'year'
        , 'month') \
    .repartition('year', 'month') \
    .rdd


songplays = spark.createDataFrame(songplays_rdd, songplays_schema)

songplays.show(2)

+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|songplay_id|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|year|month|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+
|          1|2018-11-21 21:56:...|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|2018|   11|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+----+-----+



In [297]:
songplays \
    .write \
    .partitionBy('year', 'month') \
    .mode('overwrite') \
    .parquet('tables/songplays/songplays.parquet')